In [1]:
import pandas as pd

* Задание 1
Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [30]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [31]:
p_tablle = ratings.pivot_table(index = 'userId', values = 'rating', aggfunc = 'count')
p_tablle.head() #кол-во оценок каждого userId 

,rating
userId,
1,20
2,76
3,51
4,204
5,100


In [32]:
p_tablle.reset_index()

,userId,rating
0,1,20
1,2,76
2,3,51
3,4,204
4,5,100
...,...,...
666,667,68
667,668,20
668,669,37
669,670,31


In [33]:
p_tablle2 = ratings.pivot_table(index = 'userId', values = 'timestamp', aggfunc = [max, min])
p_tablle2.head() # max и min значение timestamp для каждого userId

,max,min
,timestamp,timestamp
userId,,
1,1260759205,1260759108
2,835356246,835355395
3,1298932787,1298861589
4,949982274,949778714
5,1163375145,1163373044


In [36]:
def lt_users(row):
    """Считает время жизни пользователя"""
    row['LT'] = int(row['max'] - row['min'])
    return row

In [37]:
p_tablle3 = p_tablle2.apply(lt_users, axis=1) # добавление времени жизни пользователя
p_tablle3

,max,min,LT
,timestamp,timestamp,
userId,,,
1,1260759205,1260759108,97
2,835356246,835355395,851
3,1298932787,1298861589,71198
4,949982274,949778714,203560
5,1163375145,1163373044,2101
...,...,...,...
667,847272234,847271220,1014
668,993613478,993613196,282


In [38]:
p_tablle3.reset_index()

,userId,max,min,LT
,,timestamp,timestamp,
0,1,1260759205,1260759108,97
1,2,835356246,835355395,851
2,3,1298932787,1298861589,71198
3,4,949982274,949778714,203560
4,5,1163375145,1163373044,2101
...,...,...,...,...
666,667,847272234,847271220,1014
667,668,993613478,993613196,282
668,669,1015829766,1015829081,685


In [52]:
p_table_joined = p_tablle.join(p_tablle3)
p_table_joined

C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


,rating,"(max, timestamp)","(min, timestamp)","(LT, )"
userId,,,,
1,20,1260759205,1260759108,97
2,76,835356246,835355395,851
3,51,1298932787,1298861589,71198
4,204,949982274,949778714,203560
5,100,1163375145,1163373044,2101
...,...,...,...,...
667,68,847272234,847271220,1014
668,20,993613478,993613196,282
669,37,1015829766,1015829081,685


In [53]:
p_table_filtred = p_table_joined.query('rating' ">= 100") # фильтр на кол-во оценок
p_table_filtred

,rating,"(max, timestamp)","(min, timestamp)","(LT, )"
userId,,,,
4,204,949982274,949778714,203560
5,100,1163375145,1163373044,2101
8,116,1154474527,1154389340,85187
15,1700,1469330735,997937239,471393496
17,363,1127476640,1127468587,8053
...,...,...,...,...
656,128,986244044,986240991,3053
659,142,866207451,834598040,31609411
664,519,1441911722,1343731283,98180439


In [54]:
p_table_filtred.iloc[-1].mean() # среднее время жизни

537392396.25

* Задание 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [42]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [43]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [44]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [45]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [46]:
df_1 = rzd.merge(auto, how='outer', on='client_id')
df_1

,client_id,rzd_revenue,auto_revenue
0,111,1093.0,NaN
1,112,2810.0,NaN
2,113,10283.0,57483.0
3,114,5774.0,83.0
4,115,981.0,912.0
5,116,NaN,4834.0
6,117,NaN,98.0


In [47]:
df_2 = df_1.merge(air, how='outer', on='client_id') #таблицa с тремя типами выручки для каждого client_id без адреса клиента
df_2

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [48]:
df_3 = df_2.merge(client_base, how='left') #таблица с адресами и выручкой для client_id
df_3

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4
